# Depth Estimator Model

### Dependencies

In [28]:
import cv2
import joblib
import pickle
from PIL import Image
import numpy as np
import tensorflow as tf
import xgboost as xg
from datasets import load_dataset
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from tensorflow.keras.applications.resnet50 import preprocess_input

### Loading the Dataset 

In [46]:
ds = load_dataset("0jl/NYUv2", trust_remote_code=True) 
image_paths = ds['train']['image']
depths = ds['train']['depth']

### Extracting features

In [47]:
resnet = ResNet50(include_top=False,weights="imagenet",pooling="avg",input_shape=(224,224,3))

In [48]:
type(image_paths)

list

In [44]:
features_matrix = []
for path in image_paths:
    img = path.resize((224,224))
    img = np.array(img)
    img = np.expand_dims(img,axis = 0)
    img = img.astype("float32")
    img = preprocess_input(img)
    features_matrix.append(
        resnet.predict(img).flatten()
    )

1/1 [==============================] - 0s 15ms/step


### Preparing the Data

In [49]:
depths = np.array(depths)
depths = depths.mean(axis=(1, 2))
depths

array([2.87956975, 1.75641781, 1.79311199, ..., 2.61118133, 3.43427432,
       3.05379853])

In [50]:
X_train,X_test,y_train,y_test = train_test_split(features_matrix,depths)

print(y_train.shape)
print(y_test.shape)

(1086,)
(363,)


### Building the final model

In [51]:
model = xg.XGBRegressor(objective="reg:squarederror",device = "gpu")
model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='gpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [52]:
print("R^2 Score:", model.score(X_test, y_test))

R^2 Score: 0.5836899871121353


### Hyperparameter Tuning

In [53]:
model = xg.XGBRegressor(objective="reg:squarederror",device = "gpu")
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='gpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [54]:
param_grid = {
    "learning_rate": [0.01,0.1,0.2],
    "max_depth": [3,6,9],
    "subsample": [0.7,0.8,0.9],
    "colsample_bytree":[0.7,0.8,0.9],
    "colsample_bylevel":[0.7,0.8,0.9],
    "lambda":[0.5,1,2],
    "alpha":[2,3,4],
    "n_estimators":[100,200,300]
}

In [55]:
rsv = RandomizedSearchCV(estimator=model,param_distributions=param_grid,verbose=1,n_iter=10,n_jobs=1)
rsv.fit(X_train,y_train)    

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device='gpu',
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None...
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_jobs=1,
                   param_distributions={'alpha': [2, 3, 4],
                                        'colsample_bylevel': [0.7, 0.8, 0.9],
                                        'colsample_bytree': [0.7, 0.8, 0.9],
                                        'lambda': [0.5, 1, 2],
                                        'learning_rate': [0.01, 0.1, 0.2],
                                        'max_depth': [3, 6, 9],
                                        'n_estimators': [100, 200, 300],
                                        'subsample': [0.7, 0.8, 0.9]},
                   scoring='neg_mean_squared_error', verbose=1)

In [57]:
final_model = rsv.best_estimator_
print(final_model.score(X_test,y_test))

0.6726647370803639


### Persisting the models

In [27]:
with open("model.pkl", "wb") as f:
    pickle.dump(final_model, f)